In [3]:
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [4]:
# Cargar los datasets limpios subidos recientemente
gym = pd.read_csv("C:/Users/josen/Documents/MASTER/TFM/Datasets/gym.csv")
members = pd.read_csv("C:/Users/josen/Documents/MASTER/TFM/Datasets/members.csv")

In [5]:
# 2. Normaliza 'Level' y crea 'Type_Mapped'
type_map = {
    'Yoga':     'Stretching',
    'HIIT':     'Plyometrics',
    'Cardio':   'Cardio',
    'Strength': 'Strength'
} 
for entrenamiento in (members, gym):
    entrenamiento['Level'] = entrenamiento['Level'].replace('Expert', 'Advanced')
    entrenamiento['Type'] = entrenamiento['Type'].map(type_map).fillna(entrenamiento['Type'])

In [6]:
# Asignar IDs únicos
members['user_id'] = range(1, len(members) + 1)
gym['exercise_id'] = range(1, len(gym) + 1)
#  ESTE**

In [7]:
# Generar relaciones aleatorias: qué usuario hace qué ejercicio
records = []
for user_id in members['user_id']:
    num_exercises = random.randint(3, 6)  # Limitar a 3-6 ejercicios para ahorrar memoria
    sampled_exercises = gym.sample(n=num_exercises)
    for _, row in sampled_exercises.iterrows():
        records.append({
            'user_id': user_id,
            'exercise_id': row['exercise_id']
        })

# Crear tabla de relaciones
user_exercise_df = pd.DataFrame(records)

# Hacer merge con members
entrenamiento = user_exercise_df.merge(members, on='user_id', how='left')

# Hacer merge con gym
entrenamiento = entrenamiento.merge(gym, on='exercise_id', how='left')


In [8]:
entrenamiento.columns

Index(['user_id', 'exercise_id', 'Age', 'Gender', 'Weight_kg', 'Height_m',
       'Workout_time', 'Calories_Burned', 'Type_x', 'Fat_Percentage',
       'Frequency', 'Level_x', 'BMI', 'Title', 'Type_y', 'BodyPart',
       'Equipment', 'Level_y', 'BodyGroup'],
      dtype='object')

In [9]:
entrenamiento.drop(columns=['Type_x', 'Type_y', 'Type_Mapped_x', 'Type_Mapped_y', 'Title_y', 'BodyPart_y', 'BodyGroup_y', 'Equipment_y', 'Level_x', 'Level_y'], inplace=True)

KeyError: "['Type_Mapped_x', 'Type_Mapped_y', 'Title_y', 'BodyPart_y', 'BodyGroup_y', 'Equipment_y'] not found in axis"

In [ ]:
entrenamiento.columns

Index(['user_id', 'exercise_id', 'Age', 'Gender', 'Weight_kg', 'Height_m',
       'Workout_time', 'Calories_Burned', 'Fat_Percentage', 'Frequency', 'BMI',
       'Title_x', 'BodyPart_x', 'Equipment_x', 'BodyGroup_x', 'Type', 'Level'],
      dtype='object')

In [ ]:
unique = gym['Type'].unique()
unique

array(['Strength', 'Plyometrics', 'Cardio', 'Stretching', 'Powerlifting',
       'Strongman', 'Olympic Weightlifting'], dtype=object)

In [ ]:
renombrar = {
    'Title_x': 'Title',
    'BodyPart_x': 'BodyPart',
    'BodyGroup_x': 'BodyGroup',
    'Equipment_x': 'Equipment'
    # añade más pares según necesites
}

# 3. Aplicar el renombrado sobre el DataFrame
entrenamiento.rename(columns=renombrar, inplace=True)

In [ ]:
entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4348 entries, 0 to 4347
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          4348 non-null   int64  
 1   exercise_id      4348 non-null   int64  
 2   Age              4348 non-null   int64  
 3   Gender           4348 non-null   object 
 4   Weight_kg        4348 non-null   float64
 5   Height_m         4348 non-null   float64
 6   Workout_time     4348 non-null   float64
 7   Calories_Burned  4348 non-null   float64
 8   Fat_Percentage   4348 non-null   float64
 9   Frequency        4348 non-null   int64  
 10  BMI              4348 non-null   float64
 11  Title            4348 non-null   object 
 12  BodyPart         4348 non-null   object 
 13  Equipment        4348 non-null   object 
 14  BodyGroup        4348 non-null   object 
 15  Type             4348 non-null   object 
 16  Level            4348 non-null   object 
dtypes: float64(6),

In [ ]:
# Guardar el dataset combinado
entrenamiento.to_csv("C:/Users/josen/Documents/MASTER/TFM/SmartLifter.csv", index=False)